In [1]:
import random 
import warnings
import numpy as np 
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from pandas import DataFrame

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

VAR_SEED = 42
VAR_TESTSET_SIZE = 0.20
VAR_DIR_DATA_CLEANING = '../data/cleaning'

random.seed(VAR_SEED)
np.random.seed(VAR_SEED)
warnings.filterwarnings("ignore")

EJERCICIOS = pd.read_csv(f"{VAR_DIR_DATA_CLEANING}/ejercicios.csv", encoding="latin1")
ESTUDIANTES = pd.read_csv(f"{VAR_DIR_DATA_CLEANING}/estudiantes.csv", encoding="latin1")

print(EJERCICIOS.columns)
print(ESTUDIANTES.columns)

Index(['id_ejercicio', 'nombre', 'h1', 'h2', 'h3', 'h4', 's1', 's2', 's3',
       's4', 'k1', 'k2', 'k3', 'k4', 'hito', 'skill', 'knowledge',
       'complexity', 'complexity12', 'puntos', 'enunciado', 'dificultad',
       'score_a', 'score_d', 'score_p', 'score_s'],
      dtype='object')
Index(['id_estudiante', 'programa', 'exitosos', 'fallidos', 'solemne_1',
       'solemne_2', 'solemne_3', 'solemne_4', 'score_a', 'score_p', 'score_d',
       'score_s', 'e0', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8', 'e9',
       'e10', 'e11', 'e12', 'e13', 'e14', 'e15', 'e16', 'e17', 'e18', 'e19',
       'e20', 'e21', 'e22', 'e23', 'e24', 'e25', 'e26', 'e27', 'e28', 'e29',
       'e30', 'e31', 'e32', 'e33', 'e34', 'e35', 'e36', 'e37', 'e38', 'e39',
       'e40', 'e41', 'e42', 'e43', 'e44', 'e45', 'e46', 'e47', 'e48', 'e49',
       'e50', 'e51', 'e52'],
      dtype='object')


In [2]:
df_items = EJERCICIOS[['id_ejercicio','h1', 'h2', 'h3', 'h4', 's1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4']] 
df_users = ESTUDIANTES.copy()
df_users.head()

# División inicial en train y test
train_data, test_data = train_test_split(df_users, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)
# División adicional en train y validation
train_data, validation_data = train_test_split(train_data, test_size=VAR_TESTSET_SIZE, random_state=VAR_SEED)

# Crear escalador y codificador
scaler = MinMaxScaler()
label_encoder = LabelEncoder()

# Columnas a normalizar y codificar
columns_to_normalize = ['exitosos', 'fallidos', 'solemne_1', 'solemne_2', 'solemne_3', 'solemne_4', 'score_a', 'score_p', 'score_d', 'score_s']
column_to_encode = 'programa'

# Ajustar en el conjunto de entrenamiento
scaler.fit(train_data[columns_to_normalize])        # Ajustar el escalador
label_encoder.fit(train_data[column_to_encode])     # Ajustar el codificador

# Entrenamiento
train_data[columns_to_normalize] = scaler.transform(train_data[columns_to_normalize])
train_data[column_to_encode] = label_encoder.transform(train_data[column_to_encode])

# Validación
validation_data[columns_to_normalize] = scaler.transform(validation_data[columns_to_normalize])
validation_data[column_to_encode] = label_encoder.transform(validation_data[column_to_encode])

# Prueba
test_data[columns_to_normalize] = scaler.transform(test_data[columns_to_normalize])
test_data[column_to_encode] = label_encoder.transform(test_data[column_to_encode])

features_users_data = ['id_estudiante', 'programa', 'exitosos', 'fallidos', 'score_a', 'score_p', 'score_d', 'score_s']
features_users_inte = ['id_estudiante'] + [f"e{i}" for i in range(len(df_items))]

df_test_users = test_data[features_users_data]
df_train_users = train_data[features_users_data]
df_validation_users = validation_data[features_users_data]

df_test_interacciones = test_data[features_users_inte]
df_train_interacciones = train_data[features_users_inte]
df_validation_interacciones = validation_data[features_users_inte]


# # Filtrar estudiantes aprobados según las condiciones
estudiantes_aprovados = ESTUDIANTES[~ESTUDIANTES['id_estudiante'].isin(
    ESTUDIANTES.query("`solemne_1` < 4.0 and `solemne_2` < 4.0 and `solemne_3` < 4.0 and `solemne_4` < 4.0")['id_estudiante']
)]

# Total de ítems
n = len(EJERCICIOS)

# Últimas 'n' columnas
columnas = ESTUDIANTES.columns[-n:]

# Limpiar nombres de columnas eliminando la 'e' al inicio
columnas_limpias = columnas.str.replace('e', '').astype(int)

# Renombrar las columnas temporalmente para evitar problemas
estudiantes_aprovados.columns = list(ESTUDIANTES.columns[:-n]) + columnas_limpias.tolist()

# Calcular la suma para cada ítem y convertir a diccionario
popularidad_items = estudiantes_aprovados.iloc[:, -n:].sum(axis=0).to_dict()

In [3]:
class TwoTowerRecommenderSystem:

    def __init__(self, Two_Tower_Model, User_Tower, Item_Tower, 
                 user_input_size: int, item_input_size: int,
                 embedding_size: int = 64, dropout_rate: float = 0.5, learning_rate: float = 0.001, weight_decay: float = 1e-4, 
                 optimizer_system=optim.Adam, criterion_system=nn.BCELoss):
        
        self.model = Two_Tower_Model(user_input_size, item_input_size, embedding_size, dropout_rate, User_Tower, Item_Tower)
        self.optimizer = optimizer_system(self.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        self.criterion = criterion_system()


    def save_model(self, file_path: str) -> None:
        torch.save(self.model.state_dict(), file_path)
        print(f"[+] Modelo guardado en {file_path}")


    def load_model(self, file_path: str) -> None:
        self.model.load_state_dict(torch.load(file_path))
        self.model.eval()
        print(f"[+] Modelo cargado desde {file_path}")


    def load_data_train(self, data_items: DataFrame, data_users: DataFrame, data_interactions: DataFrame) -> None:
        self.items_inputs = torch.tensor(data_items.iloc[:, 1:].values).float()
        self.users_inputs = torch.tensor(data_users.iloc[:, 1:].values).float()
        self.interactions_inputs = torch.tensor(data_interactions.iloc[:, 1:].values).float()
        print(f"[+] Ítems cargados: {self.items_inputs.size(0)} ítems con {self.items_inputs.size(1)} características cada uno.")
        print(f"[+] Users cargados: {self.users_inputs.size(0)} users con {self.users_inputs.size(1)} características cada uno.")
        print(f"[+] Interactions cargados: {self.interactions_inputs.size(0)} interactions con {self.interactions_inputs.size(1)} características cada uno.")


    def train(self, epochs: int = 30) -> None:
        if self.items_inputs is None or self.users_inputs is None or self.interactions_inputs is None:
            raise ValueError("[-] No se cargaron datos de entrenamiento. Usa load_data_train() primero.")
        # Dimensiones de entrada
        num_users = self.users_inputs.size(0)           # Número de usuarios (n)
        num_items = self.items_inputs.size(0)           # Número de ítems (k)
        # Expandir datos de usuario e ítem para todas las combinaciones usuario-ítem
        user_input_expanded = self.users_inputs.unsqueeze(1).expand(-1, num_items, -1).reshape(-1, self.users_inputs.size(1))   # (n * k, m)
        item_input_expanded = self.items_inputs.repeat(num_users, 1)                                                            # (n * k, h)
        # Aplanar las etiquetas de interacciones para todas las combinaciones usuario-ítem
        labels = self.interactions_inputs.flatten()                                                                             # Tensor de tamaño (n * k)
        # Verificar dimensiones
        assert user_input_expanded.size(0) == item_input_expanded.size(0) == labels.size(0), \
            f"[-] Dimensiones incompatibles: user_input_expanded={user_input_expanded.size(0)}, " \
            f"[-] item_input_expanded={item_input_expanded.size(0)}, labels={labels.size(0)}"
        # Proceso de entrenamiento
        for epoch in range(epochs):
            self.optimizer.zero_grad()
            output = self.model(user_input_expanded, item_input_expanded)
            loss = self.criterion(output, labels)
            loss.backward()
            self.optimizer.step()
            print(f"[+] Epoch {epoch + 1}/{epochs} => Loss: {loss.item():.4f}")    


    def precision_at_k(self, true_labels, pred_scores, k):
        _, top_k_indices = torch.topk(pred_scores, k)                       # Índices de los top-k ítems predichos
        top_k_pred = torch.zeros_like(true_labels)                          # Inicializar predicciones binarias
        top_k_pred[top_k_indices] = 1                                       # Marcar los top-k como predichos
        num_true_positives = torch.sum(top_k_pred * true_labels).item()     # Ítems relevantes en top-k
        precision = num_true_positives / k                                  # Precisión
        return precision


    def recall_at_k(self, true_labels, pred_scores, k):
        _, top_k_indices = torch.topk(pred_scores, k)                       # Índices de los top-k ítems predichos
        top_k_pred = torch.zeros_like(true_labels)                          # Inicializar predicciones binarias
        top_k_pred[top_k_indices] = 1                                       # Marcar los top-k como predichos
        num_true_positives = torch.sum(top_k_pred * true_labels).item()     # Ítems relevantes en top-k
        num_relevant_items = torch.sum(true_labels).item()                  # Ítems relevantes reales
        recall = num_true_positives / num_relevant_items if num_relevant_items > 0 else 0
        return recall


    def ndcg_at_k(self, true_labels, pred_scores, k):
        _, top_k_indices = torch.topk(pred_scores, k)                                                       # Índices de los top-k ítems predichos
        ideal_sorted_labels = torch.sort(true_labels, descending=True)[0][:k]                               # Relevancias ideales ordenadas        
        dcg = torch.sum(true_labels[top_k_indices] / torch.log2(torch.arange(2, k + 2).float())).item()     # DCG (Discounted Cumulative Gain)
        ideal_dcg = torch.sum(ideal_sorted_labels / torch.log2(torch.arange(2, k + 2).float())).item()      # IDCG (Ideal Discounted Cumulative Gain)
        ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0
        return ndcg


    def evaluate(self, val_users: DataFrame, val_interactions: DataFrame, k: int = 10) -> None:
        # Convertir datos a tensores
        user_inputs = torch.tensor(val_users.iloc[:, 1:].values).float()
        interactions = torch.tensor(val_interactions.iloc[:, 1:].values).float()
        
        # Dimensiones
        num_items = self.items_inputs.size(0)
        num_users = user_inputs.size(0)
        
        # Expandir para todas las combinaciones usuario-ítem
        user_input_expanded = user_inputs.unsqueeze(1).expand(-1, num_items, -1).reshape(-1, user_inputs.size(1))
        item_input_expanded = self.items_inputs.repeat(num_users, 1)
        
        # Etiquetas reales
        labels = interactions
        
        # Predicciones del modelo
        self.model.eval()
        with torch.no_grad():
            output = self.model(user_input_expanded, item_input_expanded).reshape(num_users, num_items)

        # Calcular métricas
        precisions, recalls, ndcgs = [], [], []
        for user_idx in range(num_users):
            true_labels = labels[user_idx]
            pred_scores = output[user_idx]

            precisions.append(self.precision_at_k(true_labels, pred_scores, k))
            recalls.append(self.recall_at_k(true_labels, pred_scores, k))
            ndcgs.append(self.ndcg_at_k(true_labels, pred_scores, k))

        # Promediar métricas
        mean_precision = sum(precisions) / num_users
        mean_recall = sum(recalls) / num_users
        mean_ndcg = sum(ndcgs) / num_users
        print(f"[+] Evaluation Results => Precision@{k}: {mean_precision:.4f}, Recall@{k}: {mean_recall:.4f}, NDCG@{k}: {mean_ndcg:.4f}")


    def recommend(self, user_features: DataFrame, interacted_items: list[int], top_k: int = 10) -> list[tuple]:
        if self.items_inputs is None:
            raise ValueError("[-] No se cargaron datos de entrenamiento. Usa load_data_train() primero.")

        user_features = torch.tensor(user_features.iloc[:, 1:].values).float()

        self.model.eval()
        with torch.no_grad():
            user_embedding = self.model.user_tower(user_features.unsqueeze(0))                      # Generar embedding del usuario     # (1, embedding_size)
            item_embeddings = self.model.item_tower(self.items_inputs)                              # Generar embeddings de los ítems   # (num_items, embedding_size)
            scores = torch.matmul(item_embeddings, user_embedding.squeeze().unsqueeze(1)).squeeze() # Calcular los scores
            
            if not interacted_items: # Validar `interacted_items`
                print("[-] Warning: La lista interacted_items está vacía.")
            else:
                for idx in interacted_items:
                    if idx < 0 or idx >= len(scores):
                        raise ValueError(f"Índice fuera de rango: {idx}")

            for idx in interacted_items:                                                                                # Penalizar ítems ya interactuados
                scores[idx] = float('-inf')                                                                             # Penalizar ítems interactuados con -inf

            top_k_scores, top_k_indices = torch.topk(scores, top_k)                                                     # Obtener los top-k ítems
            recommendations = [ (idx, score) for idx, score in zip(top_k_indices.tolist(), top_k_scores.tolist()) ]     # Generar recomendaciones
            return recommendations


    def evaluate_relevance(self, recommended_items: dict[int, list[int]], popularity_bias: dict[int, int]) -> None:
        # Coverage
        unique_items = set(item for items in recommended_items.values() for item in items)
        coverage = len(unique_items) / self.items_inputs.shape[0]

        min_popularity = 1 / (sum(popularity_bias.values()) + 1)  # Calcular un valor mínimo positivo para ítems sin interacciones (Evita división por cero)

        # Novelty
        novelty = 0
        total_recommendations = 0
        for items in recommended_items.values():
            for item in items:  # Obtener la popularidad del ítem, usando el mínimo si no tiene interacciones
                item_popularity = max(popularity_bias.get(item, 0), min_popularity)
                novelty += -torch.log2(torch.tensor(item_popularity))
                total_recommendations += 1
        novelty /= total_recommendations

        # Popularity Bias
        avg_popularity = 0
        for items in recommended_items.values():
            avg_popularity += sum(popularity_bias.get(item, 0) for item in items)
        avg_popularity /= total_recommendations

        # Imprimir métricas
        print(f"Coverage: {coverage:.4f}, Novelty: {novelty:.4f}, Popularity Bias: {avg_popularity:.4f}")


In [4]:
from utils.config_torres import *

In [5]:
#  ████████╗██╗    ██╗ ██████╗ ████████╗ ██████╗ ██╗    ██╗███████╗██████╗ ███╗   ███╗ ██████╗ ██████╗ ███████╗██╗     
#  ╚══██╔══╝██║    ██║██╔═══██╗╚══██╔══╝██╔═══██╗██║    ██║██╔════╝██╔══██╗████╗ ████║██╔═══██╗██╔══██╗██╔════╝██║     
#     ██║   ██║ █╗ ██║██║   ██║   ██║   ██║   ██║██║ █╗ ██║█████╗  ██████╔╝██╔████╔██║██║   ██║██║  ██║█████╗  ██║     
#     ██║   ██║███╗██║██║   ██║   ██║   ██║   ██║██║███╗██║██╔══╝  ██╔══██╗██║╚██╔╝██║██║   ██║██║  ██║██╔══╝  ██║     
#     ██║   ╚███╔███╔╝╚██████╔╝   ██║   ╚██████╔╝╚███╔███╔╝███████╗██║  ██║██║ ╚═╝ ██║╚██████╔╝██████╔╝███████╗███████╗
#     ╚═╝    ╚══╝╚══╝  ╚═════╝    ╚═╝    ╚═════╝  ╚══╝╚══╝ ╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝ ╚═════╝ ╚═════╝ ╚══════╝╚══════╝


# V1
recomendador_v1_1 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModel, 
    User_Tower=UserTowerV1,
    Item_Tower=ItemTowerV1,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v1_1.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v1_1.train()


# V2
recomendador_v1_2 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModel, 
    User_Tower=UserTowerV2,
    Item_Tower=ItemTowerV2,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v1_2.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v1_2.train()


# V3
recomendador_v1_3 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModel, 
    User_Tower=UserTowerV3,
    Item_Tower=ItemTowerV3,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v1_3.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v1_3.train()

[+] Ítems cargados: 53 ítems con 12 características cada uno.
[+] Users cargados: 698 users con 7 características cada uno.
[+] Interactions cargados: 698 interactions con 53 características cada uno.
[+] Epoch 1/30 => Loss: 0.6424
[+] Epoch 2/30 => Loss: 0.5488
[+] Epoch 3/30 => Loss: 0.5139
[+] Epoch 4/30 => Loss: 0.5002
[+] Epoch 5/30 => Loss: 0.4884
[+] Epoch 6/30 => Loss: 0.4752
[+] Epoch 7/30 => Loss: 0.4523
[+] Epoch 8/30 => Loss: 0.4339
[+] Epoch 9/30 => Loss: 0.4129
[+] Epoch 10/30 => Loss: 0.3998
[+] Epoch 11/30 => Loss: 0.3918
[+] Epoch 12/30 => Loss: 0.3883
[+] Epoch 13/30 => Loss: 0.3817
[+] Epoch 14/30 => Loss: 0.3747
[+] Epoch 15/30 => Loss: 0.3671
[+] Epoch 16/30 => Loss: 0.3623
[+] Epoch 17/30 => Loss: 0.3566
[+] Epoch 18/30 => Loss: 0.3536
[+] Epoch 19/30 => Loss: 0.3506
[+] Epoch 20/30 => Loss: 0.3501
[+] Epoch 21/30 => Loss: 0.3474
[+] Epoch 22/30 => Loss: 0.3443
[+] Epoch 23/30 => Loss: 0.3405
[+] Epoch 24/30 => Loss: 0.3375
[+] Epoch 25/30 => Loss: 0.3337
[+] Epoc

In [ ]:
recomendador_v1_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v1_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v1_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
print()
recomendador_v1_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v1_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v1_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
print()
recomendador_v1_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v1_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v1_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)

[+] Evaluation Results => Precision@5: 0.7646, Recall@5: 0.3684, NDCG@5: 0.8085
[+] Evaluation Results => Precision@5: 0.7474, Recall@5: 0.3791, NDCG@5: 0.7863
[+] Evaluation Results => Precision@5: 0.7474, Recall@5: 0.3791, NDCG@5: 0.8040

[+] Evaluation Results => Precision@10: 0.6680, Recall@10: 0.6016, NDCG@10: 0.7804
[+] Evaluation Results => Precision@10: 0.7371, Recall@10: 0.7004, NDCG@10: 0.8459
[+] Evaluation Results => Precision@10: 0.7034, Recall@10: 0.6664, NDCG@10: 0.8208

[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8426
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8585
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8505


In [7]:
#  ████████╗██╗    ██╗ ██████╗ ████████╗ ██████╗ ██╗    ██╗███████╗██████╗ ███╗   ███╗ ██████╗ ██████╗ ███████╗██╗    ██╗   ██╗ ██╗
#  ╚══██╔══╝██║    ██║██╔═══██╗╚══██╔══╝██╔═══██╗██║    ██║██╔════╝██╔══██╗████╗ ████║██╔═══██╗██╔══██╗██╔════╝██║    ██║   ██║███║
#     ██║   ██║ █╗ ██║██║   ██║   ██║   ██║   ██║██║ █╗ ██║█████╗  ██████╔╝██╔████╔██║██║   ██║██║  ██║█████╗  ██║    ██║   ██║╚██║
#     ██║   ██║███╗██║██║   ██║   ██║   ██║   ██║██║███╗██║██╔══╝  ██╔══██╗██║╚██╔╝██║██║   ██║██║  ██║██╔══╝  ██║    ╚██╗ ██╔╝ ██║
#     ██║   ╚███╔███╔╝╚██████╔╝   ██║   ╚██████╔╝╚███╔███╔╝███████╗██║  ██║██║ ╚═╝ ██║╚██████╔╝██████╔╝███████╗███████╗╚████╔╝  ██║
#     ╚═╝    ╚══╝╚══╝  ╚═════╝    ╚═╝    ╚═════╝  ╚══╝╚══╝ ╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝ ╚═════╝ ╚═════╝ ╚══════╝╚══════╝ ╚═══╝   ╚═╝

# V1
recomendador_v2_1 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModelV1, 
    User_Tower=UserTowerV1,
    Item_Tower=ItemTowerV1,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v2_1.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v2_1.train()


# V2
recomendador_v2_2 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModelV1, 
    User_Tower=UserTowerV2,
    Item_Tower=ItemTowerV2,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v2_2.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v2_2.train()


# V3
recomendador_v2_3 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModelV1, 
    User_Tower=UserTowerV3,
    Item_Tower=ItemTowerV3,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v2_3.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v2_3.train()

[+] Ítems cargados: 53 ítems con 12 características cada uno.
[+] Users cargados: 698 users con 7 características cada uno.
[+] Interactions cargados: 698 interactions con 53 características cada uno.
[+] Epoch 1/30 => Loss: 0.7059
[+] Epoch 2/30 => Loss: 0.6751
[+] Epoch 3/30 => Loss: 0.6473
[+] Epoch 4/30 => Loss: 0.6246
[+] Epoch 5/30 => Loss: 0.6067
[+] Epoch 6/30 => Loss: 0.5917
[+] Epoch 7/30 => Loss: 0.5796
[+] Epoch 8/30 => Loss: 0.5707
[+] Epoch 9/30 => Loss: 0.5630
[+] Epoch 10/30 => Loss: 0.5569
[+] Epoch 11/30 => Loss: 0.5513
[+] Epoch 12/30 => Loss: 0.5468
[+] Epoch 13/30 => Loss: 0.5431
[+] Epoch 14/30 => Loss: 0.5397
[+] Epoch 15/30 => Loss: 0.5357
[+] Epoch 16/30 => Loss: 0.5331
[+] Epoch 17/30 => Loss: 0.5303
[+] Epoch 18/30 => Loss: 0.5274
[+] Epoch 19/30 => Loss: 0.5246
[+] Epoch 20/30 => Loss: 0.5217
[+] Epoch 21/30 => Loss: 0.5192
[+] Epoch 22/30 => Loss: 0.5166
[+] Epoch 23/30 => Loss: 0.5138
[+] Epoch 24/30 => Loss: 0.5110
[+] Epoch 25/30 => Loss: 0.5080
[+] Epoc

In [8]:
recomendador_v2_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v2_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v2_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
print()
recomendador_v2_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v2_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v2_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
print()
recomendador_v2_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v2_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v2_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)

[+] Evaluation Results => Precision@5: 0.7394, Recall@5: 0.3650, NDCG@5: 0.7562
[+] Evaluation Results => Precision@5: 0.7474, Recall@5: 0.3791, NDCG@5: 0.7753
[+] Evaluation Results => Precision@5: 0.7314, Recall@5: 0.3338, NDCG@5: 0.7770

[+] Evaluation Results => Precision@10: 0.6669, Recall@10: 0.6079, NDCG@10: 0.7842
[+] Evaluation Results => Precision@10: 0.7149, Recall@10: 0.6740, NDCG@10: 0.8258
[+] Evaluation Results => Precision@10: 0.6920, Recall@10: 0.6151, NDCG@10: 0.7820

[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8074
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8457
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8304


In [9]:
#  ████████╗██╗    ██╗ ██████╗ ████████╗ ██████╗ ██╗    ██╗███████╗██████╗ ███╗   ███╗ ██████╗ ██████╗ ███████╗██╗    ██╗   ██╗██████╗ 
#  ╚══██╔══╝██║    ██║██╔═══██╗╚══██╔══╝██╔═══██╗██║    ██║██╔════╝██╔══██╗████╗ ████║██╔═══██╗██╔══██╗██╔════╝██║    ██║   ██║╚════██╗
#     ██║   ██║ █╗ ██║██║   ██║   ██║   ██║   ██║██║ █╗ ██║█████╗  ██████╔╝██╔████╔██║██║   ██║██║  ██║█████╗  ██║    ██║   ██║ █████╔╝
#     ██║   ██║███╗██║██║   ██║   ██║   ██║   ██║██║███╗██║██╔══╝  ██╔══██╗██║╚██╔╝██║██║   ██║██║  ██║██╔══╝  ██║    ╚██╗ ██╔╝██╔═══╝ 
#     ██║   ╚███╔███╔╝╚██████╔╝   ██║   ╚██████╔╝╚███╔███╔╝███████╗██║  ██║██║ ╚═╝ ██║╚██████╔╝██████╔╝███████╗███████╗╚████╔╝ ███████╗
#     ╚═╝    ╚══╝╚══╝  ╚═════╝    ╚═╝    ╚═════╝  ╚══╝╚══╝ ╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝ ╚═════╝ ╚═════╝ ╚══════╝╚══════╝ ╚═══╝  ╚══════╝

# V1
recomendador_v3_1 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModelV2, 
    User_Tower=UserTowerV1,
    Item_Tower=ItemTowerV1,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v3_1.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v3_1.train()


# V2
recomendador_v3_2 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModelV2, 
    User_Tower=UserTowerV2,
    Item_Tower=ItemTowerV2,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v3_2.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v3_2.train()


# V3
recomendador_v3_3 = TwoTowerRecommenderSystem(
    Two_Tower_Model=TwoTowerModelV2, 
    User_Tower=UserTowerV3,
    Item_Tower=ItemTowerV3,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v3_3.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v3_3.train()


[+] Ítems cargados: 53 ítems con 12 características cada uno.
[+] Users cargados: 698 users con 7 características cada uno.
[+] Interactions cargados: 698 interactions con 53 características cada uno.
[+] Epoch 1/30 => Loss: 0.7172
[+] Epoch 2/30 => Loss: 0.6917
[+] Epoch 3/30 => Loss: 0.6684
[+] Epoch 4/30 => Loss: 0.6462
[+] Epoch 5/30 => Loss: 0.6254
[+] Epoch 6/30 => Loss: 0.6052
[+] Epoch 7/30 => Loss: 0.5863
[+] Epoch 8/30 => Loss: 0.5683
[+] Epoch 9/30 => Loss: 0.5529
[+] Epoch 10/30 => Loss: 0.5405
[+] Epoch 11/30 => Loss: 0.5284
[+] Epoch 12/30 => Loss: 0.5213
[+] Epoch 13/30 => Loss: 0.5168
[+] Epoch 14/30 => Loss: 0.5141
[+] Epoch 15/30 => Loss: 0.5136
[+] Epoch 16/30 => Loss: 0.5144
[+] Epoch 17/30 => Loss: 0.5101
[+] Epoch 18/30 => Loss: 0.5053
[+] Epoch 19/30 => Loss: 0.5010
[+] Epoch 20/30 => Loss: 0.4951
[+] Epoch 21/30 => Loss: 0.4865
[+] Epoch 22/30 => Loss: 0.4769
[+] Epoch 23/30 => Loss: 0.4674
[+] Epoch 24/30 => Loss: 0.4595
[+] Epoch 25/30 => Loss: 0.4501
[+] Epoc

In [10]:
recomendador_v3_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v3_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v3_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
print()
recomendador_v3_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v3_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v3_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
print()
recomendador_v3_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v3_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v3_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)

[+] Evaluation Results => Precision@5: 0.7177, Recall@5: 0.3517, NDCG@5: 0.7780
[+] Evaluation Results => Precision@5: 0.6651, Recall@5: 0.2847, NDCG@5: 0.6875
[+] Evaluation Results => Precision@5: 0.6651, Recall@5: 0.2847, NDCG@5: 0.6875

[+] Evaluation Results => Precision@10: 0.7166, Recall@10: 0.6735, NDCG@10: 0.7996
[+] Evaluation Results => Precision@10: 0.6457, Recall@10: 0.5824, NDCG@10: 0.6516
[+] Evaluation Results => Precision@10: 0.6457, Recall@10: 0.5824, NDCG@10: 0.6516

[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8297
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.7149
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.7149


In [11]:
#  ██╗███╗   ██╗████████╗████████╗ ██████╗ ██╗    ██╗███████╗██████╗ 
#  ██║████╗  ██║╚══██╔══╝╚══██╔══╝██╔═══██╗██║    ██║██╔════╝██╔══██╗
#  ██║██╔██╗ ██║   ██║      ██║   ██║   ██║██║ █╗ ██║█████╗  ██████╔╝
#  ██║██║╚██╗██║   ██║      ██║   ██║   ██║██║███╗██║██╔══╝  ██╔══██╗
#  ██║██║ ╚████║   ██║      ██║   ╚██████╔╝╚███╔███╔╝███████╗██║  ██║
#  ╚═╝╚═╝  ╚═══╝   ╚═╝      ╚═╝    ╚═════╝  ╚══╝╚══╝ ╚══════╝╚═╝  ╚═╝

# V1
recomendador_v4_1 = TwoTowerRecommenderSystem(
    Two_Tower_Model=IntTower, 
    User_Tower=UserTowerV1,
    Item_Tower=ItemTowerV1,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v4_1.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v4_1.train()


# V2
recomendador_v4_2 = TwoTowerRecommenderSystem(
    Two_Tower_Model=IntTower, 
    User_Tower=UserTowerV2,
    Item_Tower=ItemTowerV2,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v4_2.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v4_2.train()


# V3
recomendador_v4_3 = TwoTowerRecommenderSystem(
    Two_Tower_Model=IntTower, 
    User_Tower=UserTowerV3,
    Item_Tower=ItemTowerV3,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v4_3.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v4_3.train()


[+] Ítems cargados: 53 ítems con 12 características cada uno.
[+] Users cargados: 698 users con 7 características cada uno.
[+] Interactions cargados: 698 interactions con 53 características cada uno.
[+] Epoch 1/30 => Loss: 0.7006
[+] Epoch 2/30 => Loss: 0.6664
[+] Epoch 3/30 => Loss: 0.6356
[+] Epoch 4/30 => Loss: 0.6068
[+] Epoch 5/30 => Loss: 0.5807
[+] Epoch 6/30 => Loss: 0.5573
[+] Epoch 7/30 => Loss: 0.5370
[+] Epoch 8/30 => Loss: 0.5226
[+] Epoch 9/30 => Loss: 0.5135
[+] Epoch 10/30 => Loss: 0.5102
[+] Epoch 11/30 => Loss: 0.5111
[+] Epoch 12/30 => Loss: 0.5117
[+] Epoch 13/30 => Loss: 0.5102
[+] Epoch 14/30 => Loss: 0.5058
[+] Epoch 15/30 => Loss: 0.4978
[+] Epoch 16/30 => Loss: 0.4876
[+] Epoch 17/30 => Loss: 0.4767
[+] Epoch 18/30 => Loss: 0.4669
[+] Epoch 19/30 => Loss: 0.4567
[+] Epoch 20/30 => Loss: 0.4478
[+] Epoch 21/30 => Loss: 0.4387
[+] Epoch 22/30 => Loss: 0.4308
[+] Epoch 23/30 => Loss: 0.4226
[+] Epoch 24/30 => Loss: 0.4138
[+] Epoch 25/30 => Loss: 0.4044
[+] Epoc

In [12]:
recomendador_v4_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v4_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v4_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
print()
recomendador_v4_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v4_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v4_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
print()
recomendador_v4_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v4_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v4_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)

[+] Evaluation Results => Precision@5: 0.7554, Recall@5: 0.3808, NDCG@5: 0.7681
[+] Evaluation Results => Precision@5: 0.5223, Recall@5: 0.2145, NDCG@5: 0.4666
[+] Evaluation Results => Precision@5: 0.6034, Recall@5: 0.2852, NDCG@5: 0.7039

[+] Evaluation Results => Precision@10: 0.6457, Recall@10: 0.5824, NDCG@10: 0.7549
[+] Evaluation Results => Precision@10: 0.5737, Recall@10: 0.4729, NDCG@10: 0.5462
[+] Evaluation Results => Precision@10: 0.6457, Recall@10: 0.5824, NDCG@10: 0.7691

[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8033
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.6783
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8362


In [13]:
#  ██████╗ ███████╗██╗   ██╗ ██████╗ ███╗   ██╗██████╗     ████████╗██╗    ██╗ ██████╗    ████████╗ ██████╗ ██╗    ██╗███████╗██████╗ 
#  ██╔══██╗██╔════╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔══██╗    ╚══██╔══╝██║    ██║██╔═══██╗   ╚══██╔══╝██╔═══██╗██║    ██║██╔════╝██╔══██╗
#  ██████╔╝█████╗   ╚████╔╝ ██║   ██║██╔██╗ ██║██║  ██║       ██║   ██║ █╗ ██║██║   ██║█████╗██║   ██║   ██║██║ █╗ ██║█████╗  ██████╔╝
#  ██╔══██╗██╔══╝    ╚██╔╝  ██║   ██║██║╚██╗██║██║  ██║       ██║   ██║███╗██║██║   ██║╚════╝██║   ██║   ██║██║███╗██║██╔══╝  ██╔══██╗
#  ██████╔╝███████╗   ██║   ╚██████╔╝██║ ╚████║██████╔╝       ██║   ╚███╔███╔╝╚██████╔╝      ██║   ╚██████╔╝╚███╔███╔╝███████╗██║  ██║
#  ╚═════╝ ╚══════╝   ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚═════╝        ╚═╝    ╚══╝╚══╝  ╚═════╝       ╚═╝    ╚═════╝  ╚══╝╚══╝ ╚══════╝╚═╝  ╚═╝

# V1
recomendador_v5_1 = TwoTowerRecommenderSystem(
    Two_Tower_Model=SparseTwoTower, 
    User_Tower=UserTowerV1,
    Item_Tower=ItemTowerV1,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v5_1.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v5_1.train()


# V2
recomendador_v5_2 = TwoTowerRecommenderSystem(
    Two_Tower_Model=SparseTwoTower, 
    User_Tower=UserTowerV2,
    Item_Tower=ItemTowerV2,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v5_2.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v5_2.train()


# V3
recomendador_v5_3 = TwoTowerRecommenderSystem(
    Two_Tower_Model=SparseTwoTower, 
    User_Tower=UserTowerV3,
    Item_Tower=ItemTowerV3,
    user_input_size=df_train_users.shape[1] - 1,
    item_input_size=df_items.shape[1] - 1,
)
recomendador_v5_3.load_data_train(data_items=df_items, data_users=df_train_users, data_interactions=df_train_interacciones)
recomendador_v5_3.train()

[+] Ítems cargados: 53 ítems con 12 características cada uno.
[+] Users cargados: 698 users con 7 características cada uno.
[+] Interactions cargados: 698 interactions con 53 características cada uno.
[+] Epoch 1/30 => Loss: 0.6038
[+] Epoch 2/30 => Loss: 0.5246
[+] Epoch 3/30 => Loss: 0.4891
[+] Epoch 4/30 => Loss: 0.4772
[+] Epoch 5/30 => Loss: 0.4644
[+] Epoch 6/30 => Loss: 0.4486
[+] Epoch 7/30 => Loss: 0.4267
[+] Epoch 8/30 => Loss: 0.4042
[+] Epoch 9/30 => Loss: 0.3817
[+] Epoch 10/30 => Loss: 0.3695
[+] Epoch 11/30 => Loss: 0.3630
[+] Epoch 12/30 => Loss: 0.3594
[+] Epoch 13/30 => Loss: 0.3608
[+] Epoch 14/30 => Loss: 0.3562
[+] Epoch 15/30 => Loss: 0.3488
[+] Epoch 16/30 => Loss: 0.3462
[+] Epoch 17/30 => Loss: 0.3409
[+] Epoch 18/30 => Loss: 0.3406
[+] Epoch 19/30 => Loss: 0.3401
[+] Epoch 20/30 => Loss: 0.3336
[+] Epoch 21/30 => Loss: 0.3289
[+] Epoch 22/30 => Loss: 0.3260
[+] Epoch 23/30 => Loss: 0.3188
[+] Epoch 24/30 => Loss: 0.3148
[+] Epoch 25/30 => Loss: 0.3135
[+] Epoc

In [14]:
recomendador_v5_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v5_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
recomendador_v5_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=5)
print()
recomendador_v5_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v5_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
recomendador_v5_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=10)
print()
recomendador_v5_1.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v5_2.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)
recomendador_v5_3.evaluate(val_users=df_validation_users, val_interactions=df_validation_interacciones, k=15)

[+] Evaluation Results => Precision@5: 0.7029, Recall@5: 0.3353, NDCG@5: 0.7638
[+] Evaluation Results => Precision@5: 0.7360, Recall@5: 0.3667, NDCG@5: 0.7776
[+] Evaluation Results => Precision@5: 0.7474, Recall@5: 0.3791, NDCG@5: 0.7949

[+] Evaluation Results => Precision@10: 0.7194, Recall@10: 0.6900, NDCG@10: 0.8269
[+] Evaluation Results => Precision@10: 0.7371, Recall@10: 0.7004, NDCG@10: 0.8440
[+] Evaluation Results => Precision@10: 0.7149, Recall@10: 0.6799, NDCG@10: 0.8290

[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8504
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8564
[+] Evaluation Results => Precision@15: 0.6316, Recall@15: 0.8520, NDCG@15: 0.8523


In [15]:

recommended_items_v1_1 = {}
recommended_items_v1_2 = {}
recommended_items_v1_3 = {}


recommended_items_v2_1 = {}
recommended_items_v2_2 = {}
recommended_items_v2_3 = {}


recommended_items_v3_1 = {}
recommended_items_v3_2 = {}
recommended_items_v3_3 = {}


recommended_items_v4_1 = {}
recommended_items_v4_2 = {}
recommended_items_v4_3 = {}


recommended_items_v5_1 = {}
recommended_items_v5_2 = {}
recommended_items_v5_3 = {}


# Iterar sobre todos los usuarios en el conjunto de prueba
for user_id in df_test_users['id_estudiante']:
    
    # Obtener las características del usuario actual
    user = df_test_users[df_test_users['id_estudiante'] == user_id]
    interacted = [i for i, interaction in enumerate(df_test_interacciones[df_test_interacciones['id_estudiante'] == user_id].iloc[:, 1:].values.flatten()) if interaction == 1]
    
    # Recomendaciones
    recommendations_v1_1 = recomendador_v1_1.recommend(user, interacted, top_k=10)
    recommendations_v1_2 = recomendador_v1_2.recommend(user, interacted, top_k=10)
    recommendations_v1_3 = recomendador_v1_3.recommend(user, interacted, top_k=10)

    recommendations_v2_1 = recomendador_v2_1.recommend(user, interacted, top_k=10)
    recommendations_v2_2 = recomendador_v2_2.recommend(user, interacted, top_k=10)
    recommendations_v2_3 = recomendador_v2_3.recommend(user, interacted, top_k=10)

    recommendations_v3_1 = recomendador_v3_1.recommend(user, interacted, top_k=10)
    recommendations_v3_2 = recomendador_v3_2.recommend(user, interacted, top_k=10)
    recommendations_v3_3 = recomendador_v3_3.recommend(user, interacted, top_k=10)

    recommendations_v4_1 = recomendador_v4_1.recommend(user, interacted, top_k=10)
    recommendations_v4_2 = recomendador_v4_2.recommend(user, interacted, top_k=10)
    recommendations_v4_3 = recomendador_v4_3.recommend(user, interacted, top_k=10)

    recommendations_v5_1 = recomendador_v5_1.recommend(user, interacted, top_k=10)
    recommendations_v5_2 = recomendador_v5_2.recommend(user, interacted, top_k=10)
    recommendations_v5_3 = recomendador_v5_3.recommend(user, interacted, top_k=10)

    # Items recomendados

    recommended_items_v1_1[user_id] = [rec[0] for rec in recommendations_v1_1] 
    recommended_items_v1_2[user_id] = [rec[0] for rec in recommendations_v1_2]
    recommended_items_v1_3[user_id] = [rec[0] for rec in recommendations_v1_3]

    recommended_items_v2_1[user_id] = [rec[0] for rec in recommendations_v2_1] 
    recommended_items_v2_2[user_id] = [rec[0] for rec in recommendations_v2_2]
    recommended_items_v2_3[user_id] = [rec[0] for rec in recommendations_v2_3]

    recommended_items_v3_1[user_id] = [rec[0] for rec in recommendations_v3_1] 
    recommended_items_v3_2[user_id] = [rec[0] for rec in recommendations_v3_2]
    recommended_items_v3_3[user_id] = [rec[0] for rec in recommendations_v3_3]

    recommended_items_v4_1[user_id] = [rec[0] for rec in recommendations_v4_1] 
    recommended_items_v4_2[user_id] = [rec[0] for rec in recommendations_v4_2]
    recommended_items_v4_3[user_id] = [rec[0] for rec in recommendations_v4_3]

    recommended_items_v5_1[user_id] = [rec[0] for rec in recommendations_v5_1] 
    recommended_items_v5_2[user_id] = [rec[0] for rec in recommendations_v5_2]
    recommended_items_v5_3[user_id] = [rec[0] for rec in recommendations_v5_3]

In [16]:
recomendador_v1_1.evaluate_relevance(
    recommended_items=recommended_items_v1_1,
    popularity_bias=popularidad_items
)

recomendador_v1_2.evaluate_relevance(
    recommended_items=recommended_items_v1_2,
    popularity_bias=popularidad_items
)

recomendador_v1_3.evaluate_relevance(
    recommended_items=recommended_items_v1_3,
    popularity_bias=popularidad_items
)

Coverage: 0.7170, Novelty: -3.9399, Popularity Bias: 273.5753
Coverage: 0.7170, Novelty: -4.8605, Popularity Bias: 280.6068
Coverage: 0.7170, Novelty: -4.7358, Popularity Bias: 276.4434


In [17]:
recomendador_v2_1.evaluate_relevance(
    recommended_items=recommended_items_v2_1,
    popularity_bias=popularidad_items
)

recomendador_v2_2.evaluate_relevance(
    recommended_items=recommended_items_v2_2,
    popularity_bias=popularidad_items
)

recomendador_v2_3.evaluate_relevance(
    recommended_items=recommended_items_v2_3,
    popularity_bias=popularidad_items
)

Coverage: 0.6038, Novelty: -1.3579, Popularity Bias: 265.7973
Coverage: 0.5472, Novelty: 0.2485, Popularity Bias: 248.8973
Coverage: 0.5660, Novelty: -0.2651, Popularity Bias: 253.7370


In [18]:
recomendador_v3_1.evaluate_relevance(
    recommended_items=recommended_items_v3_1,
    popularity_bias=popularidad_items
)

recomendador_v3_2.evaluate_relevance(
    recommended_items=recommended_items_v3_2,
    popularity_bias=popularidad_items
)

recomendador_v3_3.evaluate_relevance(
    recommended_items=recommended_items_v3_3,
    popularity_bias=popularidad_items
)

Coverage: 0.5849, Novelty: -0.3396, Popularity Bias: 254.0420
Coverage: 0.3774, Novelty: 4.0733, Popularity Bias: 30.5146
Coverage: 0.7170, Novelty: 10.7912, Popularity Bias: 31.0721


In [19]:
recomendador_v4_1.evaluate_relevance(
    recommended_items=recommended_items_v4_1,
    popularity_bias=popularidad_items
)

recomendador_v4_2.evaluate_relevance(
    recommended_items=recommended_items_v4_2,
    popularity_bias=popularidad_items
)

recomendador_v4_3.evaluate_relevance(
    recommended_items=recommended_items_v4_3,
    popularity_bias=popularidad_items
)

Coverage: 0.3585, Novelty: 11.0126, Popularity Bias: 2.2379
Coverage: 0.5472, Novelty: -0.8016, Popularity Bias: 259.5658
Coverage: 0.3774, Novelty: 0.9374, Popularity Bias: 30.3521


In [20]:
recomendador_v5_1.evaluate_relevance(
    recommended_items=recommended_items_v5_1,
    popularity_bias=popularidad_items
)

recomendador_v5_2.evaluate_relevance(
    recommended_items=recommended_items_v5_2,
    popularity_bias=popularidad_items
)

recomendador_v5_3.evaluate_relevance(
    recommended_items=recommended_items_v5_3,
    popularity_bias=popularidad_items
)

Coverage: 0.7358, Novelty: 1.4648, Popularity Bias: 223.2032
Coverage: 0.2453, Novelty: 13.3012, Popularity Bias: 0.0000
Coverage: 0.3585, Novelty: 10.9551, Popularity Bias: 2.5854
